In [1]:
import os
import tkinter as tk
from tkinter import filedialog
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

In [2]:
# Función para leer el archivo y devolver su contenido
def leer_archivo(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        return f.read()

# Función para escribir datos en un archivo
def escribir_archivo(ruta_archivo, datos):
    with open(ruta_archivo, 'wb') as f:
        f.write(datos)


In [3]:
def cifrar_archivo(ruta_archivo, clave_tamaño, ruta_salida, ruta_claves):
    datos = leer_archivo(ruta_archivo)  # Leer el archivo a cifrar
    clave = generar_clave(clave_tamaño)
    print("Clave generada:", clave)
    # Crear un cifrador AES en modo CBC
    cipher = AES.new(clave, AES.MODE_CBC)
    iv = cipher.iv  # Inicializar el vector de inicialización (IV)

    # Añadir padding (relleno) a los datos para que sean múltiplos del tamaño de bloque (16 bytes)
    datos_padded = pad(datos, AES.block_size)

    # Cifrar los datos
    ciphertext = cipher.encrypt(datos_padded)

    nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))

    # Guardar el IV y el texto cifrado en el archivo de salida
    with open(ruta_salida, 'wb') as f:
        f.write(iv)           # Guardar el IV (16 bytes)
        f.write(ciphertext)    # Guardar el texto cifrado
    guardar_clave(nombre_archivo, clave, extension, ruta_claves)
    print(f"Archivo cifrado guardado en {ruta_salida}")


In [4]:
def descifrar_archivo(ruta_archivo, ruta_claves):
    lista_claves = leer_claves(ruta_claves)
    with open(ruta_archivo, 'rb') as f:
        iv = f.read(16)        # Leer el IV (16 bytes)
        ciphertext = f.read()  # Leer el texto cifrado restante

    nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))
    
    if nombre_archivo in lista_claves:
        clave = lista_claves[nombre_archivo][0]
        extension = lista_claves[nombre_archivo][1]
        print(f"La clave para '{nombre_archivo}' es : {clave}")

    ruta_salida = filedialog.asksaveasfilename(defaultextension=extension,
                                               initialfile=nombre_archivo,
                                               filetypes=[("Todos los archivos", "*.*")])
    
    # Crear un descifrador AES en modo CBC con el mismo IV
    cipher = AES.new(clave, AES.MODE_CBC, iv=iv)

    # Descifrar los datos
    datos_padded = cipher.decrypt(ciphertext)

    try:
        # Eliminar el padding de los datos
        datos = unpad(datos_padded, AES.block_size)

        # Escribir el archivo descifrado
        escribir_archivo(ruta_salida, datos)
        print(f"Archivo descifrado guardado en {ruta_salida}")
    except ValueError:
        print("Error: el padding es incorrecto o los datos están corruptos.")


In [5]:
def generar_clave(clave_tamaño):
    if clave_tamaño == '32':
        clave = get_random_bytes(32)
    elif clave_tamaño == '24':
        clave = get_random_bytes(24)
    elif clave_tamaño == '16':
        clave = get_random_bytes(16)
    else:
        print("Opción no válida")
    return clave # AES-256 usa una clave de 256 bits (32 bytes)


In [6]:
def guardar_clave(nombre, clave, extension, ruta_claves):
    lista_claves = leer_claves(ruta_claves)
    if nombre in lista_claves:
        lista_claves[nombre] = (clave, extension)
        with open(ruta_claves,'wb') as f:
            for name in lista_claves:
                name_bytes = name.encode('utf-8')
                f.write(len(name_bytes).to_bytes(1,'big'))
                f.write(name_bytes)
                
                f.write(len(lista_claves[name][0]).to_bytes(1, 'big'))
                f.write(lista_claves[name][0])

                ext_bytes = lista_claves[name][1].encode('utf-8')
                f.write(len(ext_bytes).to_bytes(1,'big'))
                f.write(ext_bytes)
    else:
        with open(ruta_claves,'ab') as f:
            nombre_bytes = nombre.encode('utf-8')
            f.write(len(nombre_bytes).to_bytes(1,'big'))
            f.write(nombre_bytes)
            
            f.write(len(clave).to_bytes(1,'big'))
            f.write(clave)

            extension_bytes = extension.encode('utf-8')
            f.write(len(extension_bytes).to_bytes(1,'big'))
            f.write(extension_bytes)
            

In [7]:
def leer_claves(ruta_claves):
    claves_leidas = {}
    try:
        with open(ruta_claves,'rb') as f:
                while True:
                    len_nombre = f.read(1)
                    if not len_nombre:
                        break
                    len_nombre = int.from_bytes(len_nombre,'big')
                    nombre = f.read(len_nombre).decode('utf-8')
                
                    len_clave = f.read(1)
                    if not len_clave:
                        break
                    len_clave = int.from_bytes(len_clave,'big')
                    clave = f.read(len_clave)

                    len_extension = f.read(1)
                    if not len_extension:
                        break
                    len_extension = int.from_bytes(len_extension,'big')
                    extension = f.read(len_extension).decode('utf-8')
                
                    claves_leidas[nombre] = (clave, extension)
                return claves_leidas
    except FileNotFoundError:
        return claves_leidas

In [8]:
def crear_RSA(ruta_archivo_cifrado):
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    ruta_cortada = os.path.dirname(ruta_archivo_cifrado)
    ruta_privada = os.path.normpath(os.path.join(ruta_cortada, "private.pem"))
    ruta_publica = os.path.normpath(os.path.join(ruta_cortada, "public.pem"))
    
    with open(ruta_privada, "wb") as f:
        f.write(private_key)
    with open(ruta_publica, "wb") as f:
        f.write(public_key)
    

In [9]:
def habilitar_claves(ruta_archivo_cifrado):
    print("Necesitamos que nos facilite la clave privada para desencriptar el archivo de claves")
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal de Tkinter
    ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", "*.pem")] #Seleccionamos el archivo clave privada
            )
    with open(ruta_archivo, "rb") as f:
        private_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA
        
    cipher_rsa = PKCS1_OAEP.new(private_key) #Creamos el desencriptador con dicha clave privaad

    with open(ruta_archivo_cifrado, "rb") as f:
        encrypted_data = f.read() #Leemos el archivo de claves.bin 

    decrypted_data = cipher_rsa.decrypt(encrypted_data) #Desencriptamos su contenido

    with open(ruta_archivo_cifrado, "wb") as f:
        f.write(decrypted_data) #Reescribimos el archivo de claves.bin, ahora desencriptado, en su mismo archivo

    return decrypted_data

In [10]:
def encriptar_claves(ruta_archivo_cifrado):
    print("¿Desea crear un nuevo par de claves pública y privada, o desea utilizar una clave pública propia?")
    print("1. Crear nuevas")
    print("2. Utilizar mi clave")
    opcion = input("Selecciona una opción: ")
    if opcion == '1':
        crear_RSA(ruta_archivo_cifrado) #Creamos las claves pública y privada en el mismo directorio que el archivo claves.bin
        
        ruta_cortada = os.path.dirname(ruta_archivo_cifrado)
        ruta_archivo = os.path.normpath(os.path.join(ruta_cortada, "public.pem"))
        
        with open(ruta_archivo, "rb") as f:
            public_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA

        cipher_rsa = PKCS1_OAEP.new(public_key)

        with open(ruta_archivo_cifrado, "rb") as f:
            datos = f.read()
            
        encrypted_data = cipher_rsa.encrypt(datos)

        with open(ruta_archivo_cifrado, "wb") as f:
            f.write(encrypted_data)
            
    elif opcion == '2':
        print("Necesitamos que nos facilite la clave publica para encriptar el archivo de claves")
        root = tk.Tk()
        root.withdraw()  # Oculta la ventana principal de Tkinter
        ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", "*.pem")] #Seleccionamos el archivo clave publica
            )
        with open(ruta_archivo, "rb") as f:
            public_key = RSA.import_key(f.read()) #Leemos su contenido y lo importamos como clave RSA

        cipher_rsa = PKCS1_OAEP.new(public_key)

        with open(ruta_archivo_cifrado, "rb") as f:
            datos = f.read()
            
        encrypted_data = cipher_rsa.encrypt(datos)

        with open(ruta_archivo_cifrado, "wb") as f:
            f.write(encrypted_data)
    else:
        print("Opción no válida. Inténtalo de nuevo.")

In [11]:
def menu():
    print("1. Cifrar un archivo")
    print("2. Descifrar un archivo")
    print("3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)")
    print("4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES")
    print("5. Salir")
    return input("Selecciona una opción: ")


In [12]:
def main():
    lista_claves = {}
    ruta_salida = ""
    while True:
        opcion = menu()
        if opcion == '1':
            # Cifrar archivo
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo",
                filetypes=[("Todos los archivos", ".*")]
            )
            
            if ruta_archivo:
                print(f"Has seleccionado: {ruta_archivo}")
                clave = input("Introduce tamaño de clave: 32 (256), 24 (192), 16 (128)")
                nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_archivo))
                ruta_salida = filedialog.asksaveasfilename(defaultextension=".bin",
                                                           initialfile=nombre_archivo,
                                                           filetypes=[("Archivos BIN", "*.bin"),("Todos los archivos", "*.*")])
                cifrar_archivo(ruta_archivo, clave, ruta_salida, ruta_claves)
                print(f"Archivo cifrado y guardado en: {ruta_salida}")
            else:
                print("No se seleccionó ningún archivo.")
                
        elif opcion == '2':
            # Descifrar archivo
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo cifrado",
                filetypes=[("Todos los archivos", "*.bin")]
            )
            
            if ruta_archivo:
                print(f"Has seleccionado: {ruta_archivo}")
                descifrar_archivo(ruta_archivo, ruta_claves)
            else:
                print("No se seleccionó ningún archivo.")
        elif opcion == '3':
            root = tk.Tk()
            root.withdraw()  # Oculta la ventana principal de Tkinter
            ruta_archivo = filedialog.askopenfilename(
                title="Seleccionar archivo de claves",
                filetypes=[("Todos los archivos", "*.bin")]
            )
            
            if ruta_archivo:
                ruta_claves = ruta_archivo
                print("¿Este archivo está encriptado clave pública?")
                result = input("Seleccione 1 (SI) o 2 (NO)")
                if result == '1':
                    lista_claves = habilitar_claves(ruta_claves)
            else:
                print("No se seleccionó ningún archivo. Crearemos un archivo de claves nuevo...")
                ruta_claves = filedialog.asksaveasfilename(defaultextension=".bin",
                                                           initialfile="claves",
                                                           filetypes=[("Archivos BIN", "*.bin"),("Todos los archivos", "*.*")])
                with open(ruta_claves, 'wb') as f:
                    pass
        elif opcion == '4':
            encriptar_claves(ruta_claves)
        elif opcion == '5':
            print("Saliendo...")
            break
        else:
            print("Opción no válida. Inténtalo de nuevo.")


In [13]:
if __name__ == "__main__":
    main()


1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir


Selecciona una opción:  3


¿Este archivo está encriptado clave pública?


Seleccione 1 (SI) o 2 (NO) 1


Necesitamos que nos facilite la clave privada para desencriptar el archivo de claves
1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir


...

Has seleccionado: D:/EscritorioD/CarpetaPruebas/prueba2.bin
La clave para 'prueba2' es : b'\xad_\x9eS\xb8\xaf\xc8\x1a\x8bM\x0f\x88\x07e..\xe1\xe6n3\xc1"\x11\x05YB^\xff\x94\xb7(\x05'
Archivo descifrado guardado en D:/EscritorioD/CarpetaPruebas/prueba2res.txt
1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir


Selecciona una opción:  2


Has seleccionado: D:/EscritorioD/CarpetaPruebas/prueba3.bin
La clave para 'prueba3' es : b'\x0f\x80t\xc1~\xc2\xc5\xb6\xc1\xe1bkW\xecY\x99\x0fM\x07\x9c\xbf\xdd1\xb1T\x99r\x9a\xde\x90\\\xa5'
Archivo descifrado guardado en D:/EscritorioD/CarpetaPruebas/prueba3res.txt
1. Cifrar un archivo
2. Descifrar un archivo
3. Cargar archivo de claves (HACER LO PRIMERO SI YA TIENE CLAVES O PARA CREAR ARCHIVO DE CLAVES)
4. Encriptar claves (HACER LO ULTIMO SI QUIERES ENCRIPTAR LAS CLAVES
5. Salir


Selecciona una opción:  5


Saliendo...
